In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt

# 数据生成（增强和预处理）
def generate_synthetic_data(num_samples=1000, image_size=(64, 64)):
    X = np.random.rand(num_samples, *image_size)
    y = np.random.rand(num_samples, *image_size)

    # 数据增强（旋转、平移、加噪声等）
    X = np.array([np.rot90(x, np.random.randint(0, 4)) for x in X])
    y = np.array([np.rot90(y_, np.random.randint(0, 4)) for y_ in y])
    X += np.random.normal(0, 0.01, X.shape)
    return X, y

# U-Net结构
def build_unet(input_shape=(64, 64, 1)):
    inputs = layers.Input(shape=input_shape)
    # 编码器
    c1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)
    c2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)
    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)
    # Bottleneck
    bn = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p3)
    bn = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(bn)
    # 解码器
    u3 = layers.UpSampling2D((2, 2))(bn)
    concat3 = layers.Concatenate()([u3, c3])
    c4 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(concat3)
    c4 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c4)
    u2 = layers.UpSampling2D((2, 2))(c4)
    concat2 = layers.Concatenate()([u2, c2])
    c5 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(concat2)
    c5 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c5)
    u1 = layers.UpSampling2D((2, 2))(c5)
    concat1 = layers.Concatenate()([u1, c1])
    c6 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(concat1)
    c6 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c6)
    outputs = layers.Conv2D(1, (1, 1), activation='linear')(c6)
    model = models.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse', metrics=['mse', 'mae'])
    return model

# 生成数据并进行训练
X, y = generate_synthetic_data()

# 数据维度调整
X = X[..., np.newaxis]
y = y[..., np.newaxis]

# 构建U-Net模型
model = build_unet()

# 训练模型（加入早期停止机制，防止过拟合）
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# 训练过程可视化
history = model.fit(X, y, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# 绘制训练与验证损失曲线
plt.figure()
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.legend()
plt.title('Training & Validation Loss')
plt.show()

# 随机选取样本进行对比可视化
idx = np.random.randint(0, X.shape[0])
sample_input = X[idx:idx+1]
sample_true = y[idx]
sample_pred = model.predict(sample_input)[0]

plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.title('Input Data')
plt.imshow(sample_input[0, :, :, 0], cmap='gray')
plt.subplot(1, 3, 2)
plt.title('True Property')
plt.imshow(sample_true[:, :, 0], cmap='gray')
plt.subplot(1, 3, 3)
plt.title('Predicted Property')
plt.imshow(sample_pred[:, :, 0], cmap='gray')
plt.show()

# 输出最终验证集上的MSE和MAE
val_mse = history.history['val_mse'][-1]
val_mae = history.history['val_mae'][-1]
print(f'最终验证集MSE: {val_mse:.4f}, MAE: {val_mae:.4f}')